### Selenium de valores en ascenso

En este notebook se ha creado un programa el cual:
* Entra en la información de todos los jugadores
* Extrae su información histórica (Valor de mercado, equipo, edad, fecha)

In [2]:
#Servicio y driver de Chrome de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#Las opciones que vamos a tener para buscar elementos
from selenium.webdriver.common.by import By

#Para cuando queramos mandar pulsaciones de teclado
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup as bs
import requests
import datetime as dt
import time
import numpy as np
import pandas as pd
import openpyxl

Abrimos el driver

In [25]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
time.sleep(5)

Abrimos los links que necesitamos y limpiamos el DataFrame, quitando los "No veo la página" que obtuvimos anteriormente

In [4]:
df_links_mal = pd.read_excel('../data/LinksRegu_Subidas.xlsx', index_col=0)
df_links_mal

,Links
0,No veo la página
1,No veo la página
2,No veo la página
3,No veo la página
4,No veo la página
...,...
426985,No veo la página
426986,No veo la página
426987,No veo la página
426988,No veo la página


Aquí quitamos los "No veo la página"

In [5]:
df_links_normal = df_links_mal[df_links_mal['Links'] != 'No veo la página'].reset_index(drop=True)
df_links_normal.iloc[0,0]


'https://www.transfermarkt.es/enzo-fernandez/marktwertverlauf/spieler/648195'

#### Empezamos con el programa

Primero, tenemos que aceptar las cookies (y tratamos de quitar el primer pop up)

En ocasiones, al pulsar sobre la x del pop-up es posible que se abra una ventana nueva. El programa está preparado para cerrarla en caso de que eso suceda :)

In [28]:
lista_datos = []


driver.get(df_links_normal.iloc[0,0])
time.sleep(5)

try:
    iframe = driver.find_elements(By.TAG_NAME,'iframe')[-1]
    driver.switch_to.frame(iframe)
    time.sleep(1)

    aceptar = driver.find_elements(By.TAG_NAME, 'button')[-1]
    aceptar.click()
    
except:
    print('Na')
time.sleep(5)

try:
    cerrar_pop_up = driver.find_elements(By.TAG_NAME, 'span')

    for x in cerrar_pop_up:
        if "×" in x.text:
            x.click()
            time.sleep(1)
        else:
            continue
except:
    print('na')

try:
    driver.switch_to.window(driver.window_handles[0])
except:
    print('NA')

    

Na


### El Programa se ejecuta aquí

In [12]:
lista_datos = []

for url in range(len(df_links_normal['Links'])):
    try:
        driver.get(df_links_normal.iloc[url,0])
        time.sleep(5)
        
        try:
            cerrar_pop_up = driver.find_elements(By.TAG_NAME, 'span')

            for x in cerrar_pop_up:
                if "×" in x.text:
                    x.click()
                    time.sleep(2)
                else:
                    continue
            
        except:
            print('Na')
        try:
            if len(set(driver.window_handles))>1:
                driver.switch_to.window(driver.window_handles[1])
                driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            continue
        time.sleep(0.5)

        logos = driver.find_elements(By.TAG_NAME, 'image') #Primera parte
        #Aquí irán los datos
        nombre = driver.find_element(By.CLASS_NAME, 'data-header__headline-wrapper').text

        if logos != '':
        #bucle de la primera parte
            for i in logos: 
            #Action Chains sirve para crear cadenas de acción. Es algo más avanzado pero no es muy complicado
                ActionChains(driver).move_to_element(i).perform() #Movemos el cursor hacia el elemento
    
                time.sleep(0.8) #hacemos que descanse dos segundos
    
            #Quiero todos los elementos (find_elements)
                Elementos=driver.find_elements(By.CLASS_NAME, 'highcharts-tooltip') 

            #bucle en elementos
                for j in Elementos:
                    time.sleep(0.2)
                
                    print(j.text)

                #lo añado a la lista separando los valores por saltos de línes
                    lista_datos.append(j.text.split('\n') + [nombre])
    except:
        url-=1
        continue


28/11/2022
Valor de mercado: 850 mil €
Club: CSKA-Sofia
Edad : 22

17/06/2022
Valor de mercado: 750 mil €
Club: Once Caldas
Edad : 22

13/01/2022
Valor de mercado: 750 mil €
Club: Once Caldas
Edad : 22

07/06/2021
Valor de mercado: 750 mil €
Club: Once Caldas
Edad : 21

16/01/2021
Valor de mercado: 850 mil €
Club: Once Caldas
Edad : 21

08/04/2020
Valor de mercado: 775 mil €
Club: Once Caldas
Edad : 20

28/02/2020
Valor de mercado: 850 mil €
Club: Once Caldas
Edad : 20

21/06/2019
Valor de mercado: 550 mil €
Club: Once Caldas
Edad : 19

23/12/2018
Valor de mercado: 600 mil €
Club: Once Caldas
Edad : 19

09/10/2018
Valor de mercado: 150 mil €
Club: Once Caldas
Edad : 18

08/06/2018
Valor de mercado: 50 mil €
Club: Once Caldas U20
Edad : 18

22/02/2023
Valor de mercado: 850 mil €
Club: Puskás Akadémia FC
Edad : 20

07/06/2022
Valor de mercado: 750 mil €
Club: Puskás Akadémia FC
Edad : 20

07/01/2022
Valor de mercado: 600 mil €
Club: Puskás Akadémia FC
Edad : 19

11/10/2021
Valor de merc

Vemos una muestra de como se obtienen los datos

In [13]:
lista_datos

[['', '#26 Marcelino Carreazo'],
 ['28/11/2022',
  'Valor de mercado: 850 mil €',
  'Club: CSKA-Sofia',
  'Edad : 22',
  '#26 Marcelino Carreazo'],
 ['', '#26 Marcelino Carreazo'],
 ['17/06/2022',
  'Valor de mercado: 750 mil €',
  'Club: Once Caldas',
  'Edad : 22',
  '#26 Marcelino Carreazo'],
 ['', '#26 Marcelino Carreazo'],
 ['13/01/2022',
  'Valor de mercado: 750 mil €',
  'Club: Once Caldas',
  'Edad : 22',
  '#26 Marcelino Carreazo'],
 ['', '#26 Marcelino Carreazo'],
 ['07/06/2021',
  'Valor de mercado: 750 mil €',
  'Club: Once Caldas',
  'Edad : 21',
  '#26 Marcelino Carreazo'],
 ['', '#26 Marcelino Carreazo'],
 ['16/01/2021',
  'Valor de mercado: 850 mil €',
  'Club: Once Caldas',
  'Edad : 21',
  '#26 Marcelino Carreazo'],
 ['', '#26 Marcelino Carreazo'],
 ['08/04/2020',
  'Valor de mercado: 775 mil €',
  'Club: Once Caldas',
  'Edad : 20',
  '#26 Marcelino Carreazo'],
 ['', '#26 Marcelino Carreazo'],
 ['28/02/2020',
  'Valor de mercado: 850 mil €',
  'Club: Once Caldas',
  

##### Limpiamos la lista obtenida

In [14]:
lista_datos_limpia = [] #Nueva lista que NO TENDRÁ listas vacías dentro


for i in lista_datos:

    #Le digo que me añada todo lo que no sean listas vacías en una nueva lista
    if len(i) != 2 :
        lista_datos_limpia.append(i)
print(lista_datos_limpia)

[['28/11/2022', 'Valor de mercado: 850 mil €', 'Club: CSKA-Sofia', 'Edad : 22', '#26 Marcelino Carreazo'], ['17/06/2022', 'Valor de mercado: 750 mil €', 'Club: Once Caldas', 'Edad : 22', '#26 Marcelino Carreazo'], ['13/01/2022', 'Valor de mercado: 750 mil €', 'Club: Once Caldas', 'Edad : 22', '#26 Marcelino Carreazo'], ['07/06/2021', 'Valor de mercado: 750 mil €', 'Club: Once Caldas', 'Edad : 21', '#26 Marcelino Carreazo'], ['16/01/2021', 'Valor de mercado: 850 mil €', 'Club: Once Caldas', 'Edad : 21', '#26 Marcelino Carreazo'], ['08/04/2020', 'Valor de mercado: 775 mil €', 'Club: Once Caldas', 'Edad : 20', '#26 Marcelino Carreazo'], ['28/02/2020', 'Valor de mercado: 850 mil €', 'Club: Once Caldas', 'Edad : 20', '#26 Marcelino Carreazo'], ['21/06/2019', 'Valor de mercado: 550 mil €', 'Club: Once Caldas', 'Edad : 19', '#26 Marcelino Carreazo'], ['23/12/2018', 'Valor de mercado: 600 mil €', 'Club: Once Caldas', 'Edad : 19', '#26 Marcelino Carreazo'], ['09/10/2018', 'Valor de mercado: 150

#### Creamos el DataFrame

In [15]:
lista_df = []
for i in range(len(lista_datos_limpia)):
    data = {"Nombre":lista_datos_limpia[i][4],
    "Fecha de actualización": lista_datos_limpia[i][0],
    "Valor de mercado": lista_datos_limpia[i][1],
    "Club": lista_datos_limpia[i][2],
    "Edad": lista_datos_limpia[i][3],

    }
    lista_df.append(data)

In [16]:
df_new = pd.DataFrame(lista_df)
df_new.tail(15)

,Nombre,Fecha de actualización,Valor de mercado,Club,Edad
5903,#8 Cristian Álvarez,22/06/2013,Valor de mercado: 120 mil €,Club: CA Boca Juniors II,Edad : 20
5904,#8 Vladan Danilovic,28/12/2022,Valor de mercado: 525 mil €,Club: CD Nacional,Edad : 23
5905,#8 Vladan Danilovic,27/06/2022,Valor de mercado: 500 mil €,Club: CD Nacional,Edad : 22
5906,#8 Vladan Danilovic,01/04/2022,Valor de mercado: 500 mil €,Club: CD Nacional,Edad : 22
5907,#8 Vladan Danilovic,21/05/2021,Valor de mercado: 500 mil €,Club: CD Nacional,Edad : 21
5908,#8 Vladan Danilovic,28/01/2021,Valor de mercado: 500 mil €,Club: CD Nacional,Edad : 21
5909,#8 Vladan Danilovic,09/09/2020,Valor de mercado: 500 mil €,Club: FK Borac Banja Luka,Edad : 21
5910,#8 Vladan Danilovic,08/04/2020,Valor de mercado: 350 mil €,Club: FK Borac Banja Luka,Edad : 20
5911,#8 Vladan Danilovic,07/01/2020,Valor de mercado: 400 mil €,Club: FK Borac Banja Luka,Edad : 20
5912,#8 Vladan Danilovic,23/07/2019,Valor de mercado: 175 mil €,Club: FK Borac Banja Luka,Edad : 19


Limpiamos cada columna

In [17]:
df_new['Fecha de actualización'] = pd.to_datetime(df_new['Fecha de actualización'], dayfirst=True,)

In [18]:
### Limpiamos el DF
### Nombres
nombres_limpio = []
for i in df_new['Nombre']:
    nombres_limpio.append(' '.join(i.split()[1:]))

df_new['Nombre'] = nombres_limpio

In [19]:
clubes_limpio = []
for i in range(len(df_new['Club'])):
    clubes_limpio.append(" ".join(df_new['Club'][i].split()[1:]))
df_new['Club'] = clubes_limpio

In [20]:
edad_limpio = []
for i in df_new['Edad']:
    edad_limpio.append(int(i.split()[-1]))

df_new['Edad']  = edad_limpio

#### Guardo el DF

In [22]:
df_new.to_excel('../data/Selenium_Subidas.xlsx')